In [1]:
import numpy as np
import h5py

from sklearn.linear_model import LassoCV
from sklearn.metrics import r2_score

from neuropacks import PVC11
from pyuoi.linear_model import Poisson
from pyuoi.linear_model import UoI_Poisson

In [2]:
pvc = PVC11(data_path='/Users/psachdeva/data/pvc11/data/spikes_gratings/data_monkey1_gratings.mat')

In [3]:
Y = pvc.get_response_matrix(transform='square_root')

In [4]:
np.array([np.logspace(start=3, stop=0, num=30)])

array([[1000.        ,  788.04628157,  621.01694189,  489.39009185,
         385.66204212,  303.91953823,  239.502662  ,  188.73918221,
         148.73521073,  117.21022975,   92.36708572,   72.78953844,
          57.3615251 ,   45.20353656,   35.6224789 ,   28.07216204,
          22.12216291,   17.43328822,   13.73823796,   10.82636734,
           8.53167852,    6.72335754,    5.29831691,    4.17531894,
           3.29034456,    2.5929438 ,    2.04335972,    1.61026203,
           1.268961  ,    1.        ]])

In [5]:
uoipoisson = UoI_Poisson(
    lambdas=np.array([np.logspace(start=3, stop=0, num=30)]),
    normalize=False,
    n_boots_sel=30,
    n_boots_est=30,
    selection_frac=0.8,
    estimation_frac=0.8,
    solver='proximal_grad'
)

In [9]:
f = h5py.File('test.h5', 'w')
f['X'] = np.delete(Y, 3, axis=1)
f['y'] = Y[:, 3]
f.close()

In [7]:
uoipoisson.fit(np.delete(Y, 3, axis=1), Y[:, 3])

UoI_Poisson(alphas=array([1]), comm=None, copy_X=True, eps=0.001,
      estimation_frac=0.8, estimation_score='log', fit_intercept=True,
      lambdas=array([[1000.     ,  788.04628,  621.01694,  489.39009,  385.66204,
         303.91954,  239.50266,  188.73918,  148.73521,  117.21023,
          92.36709,   72.78954,   57.36153,   45.20354,   35.62248,
          28.07216,   22.12216,   17.43329,   13.73824,   10.82637,
           8.53168,    6.72336,    5.29832,    4.17532,    3.29034,
           2.59294,    2.04336,    1.61026,    1.26896,    1.     ]]),
      max_iter=None, n_boots_est=30, n_boots_sel=30, n_lambdas=48,
      normalize=False,
      random_state=<module 'numpy.random' from '/Users/psachdeva/anaconda3/lib/python3.6/site-packages/numpy/random/__init__.py'>,
      selection_frac=0.8, solver='proximal_grad', stability_selection=1.0,
      tol=None, warm_start=True)

In [12]:
np.count_nonzero(uoipoisson.coef_)

39

In [11]:
uoipoisson.intercept_

-0.26496964945858215

In [7]:
np.logspace(start=np.log10(10), stop=-1, num=30)

array([10.        ,  8.53167852,  7.27895384,  6.21016942,  5.29831691,
        4.52035366,  3.85662042,  3.29034456,  2.8072162 ,  2.39502662,
        2.04335972,  1.74332882,  1.48735211,  1.268961  ,  1.08263673,
        0.92367086,  0.78804628,  0.67233575,  0.57361525,  0.48939009,
        0.41753189,  0.35622479,  0.30391954,  0.25929438,  0.22122163,
        0.18873918,  0.1610262 ,  0.13738238,  0.11721023,  0.1       ])

In [20]:
from dask_ml.linear_model import PoissonRegression
poisson = PoissonRegression(
    penalty='l1',
    C=0.001,
    solver='proximal_grad'
)
neuron = 1
poisson.fit(np.delete(Y, 2, axis=1), Y[:, 2])
print(poisson.coef_)

[0.         0.12508932 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.03114472 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.00768411
 0.         0.         0.         0.         0.         0.
 0.         0.         0.00132911 0.         0.         0.
 0.         0.         0.         0.         0. 

In [6]:
neuron = 5
fold = 3

In [8]:
training_idx = f['LassoCV/training_folds/fold_' + str(fold)][:]
test_idx = f['LassoCV/test_folds/fold_' + str(fold)][:]

X_train = np.delete(Y[training_idx], neuron, axis=1)
y_train = Y[training_idx][:, neuron]

X_test = np.delete(Y[test_idx], neuron, axis=1)
y_test = Y[test_idx][:, neuron]

In [9]:
lasso = LassoCV(normalize=True, cv=10, max_iter=5000)
lasso.fit(X_train, y_train)

LassoCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
    max_iter=5000, n_alphas=100, n_jobs=None, normalize=True,
    positive=False, precompute='auto', random_state=None,
    selection='cyclic', tol=0.0001, verbose=False)

In [10]:
print(lasso.intercept_)
print(f['LassoCV/intercepts'][fold, neuron])

0.22239894256507453
0.22239894256507542


In [11]:
print(lasso.coef_)
print(f['LassoCV/coupling_coefs'][fold, neuron, :])

[-0.          0.16363471  0.          0.         -0.         -0.
  0.          0.01393437  0.04190941 -0.          0.01952771  0.
 -0.09287486  0.          0.00828941  0.02609144  0.          0.01845543
  0.09875288  0.          0.06589219  0.03143587  0.01697798  0.
 -0.          0.         -0.          0.04368621  0.          0.
  0.          0.          0.03344687 -0.          0.          0.
  0.          0.01829595  0.          0.03153687  0.04089385  0.01966991
  0.09854164  0.         -0.          0.02535765  0.02887826  0.02324685
  0.02146969  0.         -0.          0.03565194  0.08730009  0.02677577
  0.          0.         -0.         -0.         -0.01156719 -0.
 -0.          0.          0.          0.          0.         -0.
  0.          0.          0.          0.         -0.          0.02507811
 -0.         -0.          0.06911213  0.          0.03147191  0.01703257
  0.00654002  0.          0.          0.06061753  0.00018145  0.0644977
  0.05298076  0.          0.0050730

In [12]:
print(r2_score(y_test, np.dot(X_test, lasso.coef_) + lasso.intercept_))
print(f['LassoCV/r2'][fold, neuron])

0.6700381348012396
0.6700381348012396
